<a href="https://colab.research.google.com/github/jumriramadhan/Tugas-nlp/blob/main/Jumri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
pip install fastai==1.0.61

In [71]:
import sys
sys.path.append('/content/drive/MyDrive/course-nlp-master/')
from seq2seq import *

In [72]:
from pathlib import Path
import pickle
import torch

# Ganti dengan path folder tujuan Anda
target_path = Path("/content/drive/MyDrive/course-nlp-master/giga-fren")
target_path.mkdir(parents=True, exist_ok=True)

# 1. Simpan file dummy data_save.pkl
dummy_data = {"info": "Dummy data for testing only"}
with open(target_path / "data_save.pkl", "wb") as f:
    pickle.dump(dummy_data, f)

# 2. Simpan embedding dummy fr_emb.pth dan en_emb.pth
# Ukuran kecil supaya ringan
fr_emb = torch.randn((10, 8))
en_emb = torch.randn((10, 8))
torch.save(fr_emb, target_path / "fr_emb.pth")
torch.save(en_emb, target_path / "en_emb.pth")

print("✅ Dummy files berhasil dibuat di:", target_path)


✅ Dummy files berhasil dibuat di: /content/drive/MyDrive/course-nlp-master/giga-fren


In [73]:
from pathlib import Path
import torch

model_path = Path('/content/drive/MyDrive/course-nlp-master/giga-fren')
emb_enc = torch.load(model_path / 'fr_emb.pth')
emb_dec = torch.load(model_path / 'en_emb.pth')

print("✅ Loaded dummy embeddings successfully!")
print("Encoder shape:", emb_enc.shape)
print("Decoder shape:", emb_dec.shape)

✅ Loaded dummy embeddings successfully!
Encoder shape: torch.Size([10, 8])
Decoder shape: torch.Size([10, 8])


In [74]:
class Seq2SeqRNN_attn(nn.Module):
    def __init__(self, emb_enc, emb_dec, nh, out_sl, nl=2, bos_idx=0, pad_idx=1):
        super().__init__()
        self.nl,self.nh,self.out_sl,self.pr_force = nl,nh,out_sl,1
        self.bos_idx,self.pad_idx = bos_idx,pad_idx
        self.emb_enc,self.emb_dec = emb_enc,emb_dec
        self.emb_sz_enc,self.emb_sz_dec = emb_enc.embedding_dim,emb_dec.embedding_dim
        self.voc_sz_dec = emb_dec.num_embeddings

        self.emb_enc_drop = nn.Dropout(0.15)
        self.gru_enc = nn.GRU(self.emb_sz_enc, nh, num_layers=nl, dropout=0.25,
                              batch_first=True, bidirectional=True)
        self.out_enc = nn.Linear(2*nh, self.emb_sz_dec, bias=False)
        self.gru_dec = nn.GRU(self.emb_sz_dec + 2*nh, self.emb_sz_dec, num_layers=nl,
                              dropout=0.1, batch_first=True)
        self.out_drop = nn.Dropout(0.35)
        self.out = nn.Linear(self.emb_sz_dec, self.voc_sz_dec)
        self.out.weight.data = self.emb_dec.weight.data

        self.enc_att = nn.Linear(2*nh, self.emb_sz_dec, bias=False)
        self.hid_att = nn.Linear(self.emb_sz_dec, self.emb_sz_dec)
        self.V =  self.init_param(self.emb_sz_dec)

    def encoder(self, bs, inp):
        h = self.initHidden(bs)
        emb = self.emb_enc_drop(self.emb_enc(inp))
        enc_out, hid = self.gru_enc(emb, 2*h)
        pre_hid = hid.view(2, self.nl, bs, self.nh).permute(1,2,0,3).contiguous()
        pre_hid = pre_hid.view(self.nl, bs, 2*self.nh)
        hid = self.out_enc(pre_hid)
        return hid,enc_out

    def decoder(self, dec_inp, hid, enc_att, enc_out):
        hid_att = self.hid_att(hid[-1])
        u = torch.tanh(enc_att + hid_att[:,None])
        attn_wgts = F.softmax(u @ self.V, 1)
        ctx = (attn_wgts[...,None] * enc_out).sum(1)
        emb = self.emb_dec(dec_inp)
        outp, hid = self.gru_dec(torch.cat([emb, ctx], 1)[:,None], hid)
        outp = self.out(self.out_drop(outp[:,0]))
        return hid, outp

    def forward(self, inp, targ=None):
        bs, sl = inp.size()
        hid,enc_out = self.encoder(bs, inp)
        dec_inp = inp.new_zeros(bs).long() + self.bos_idx
        enc_att = self.enc_att(enc_out)

        res = []
        for i in range(self.out_sl):
            hid, outp = self.decoder(dec_inp, hid, enc_att, enc_out)
            res.append(outp)
            dec_inp = outp.max(1)[1]
            if (dec_inp==self.pad_idx).all(): break
            if (targ is not None) and (random.random()<self.pr_force):
                if i>=targ.shape[1]: continue
                dec_inp = targ[:,i]
        return torch.stack(res, dim=1)

    def initHidden(self, bs): return one_param(self).new_zeros(2*self.nl, bs, self.nh)
    def init_param(self, *sz): return nn.Parameter(torch.randn(sz)/math.sqrt(sz[0]))

In [75]:
import torch.nn as nn

# fr_emb dan en_emb adalah tensor
# Konversi ke nn.Embedding
emb_enc = nn.Embedding.from_pretrained(fr_emb)
emb_dec = nn.Embedding.from_pretrained(en_emb)

In [76]:
model = Seq2SeqRNN_attn(emb_enc, emb_dec, 256, 30)

In [77]:
from fastai.text import TextDataBunch
from torch.utils.data import TensorDataset, DataLoader
from fastai.basic_data import DataBunch

# Dummy input & target tensors
x_dummy = torch.randint(0, 10, (8, 5))  # 8 samples, 5 tokens each
y_dummy = torch.randint(0, 10, (8, 5))

# Buat dataset dan DataLoader
train_ds = TensorDataset(x_dummy, y_dummy)
valid_ds = TensorDataset(x_dummy, y_dummy)
train_dl = DataLoader(train_ds, batch_size=4)
valid_dl = DataLoader(valid_ds, batch_size=4)

# Buat DataBunch
data = DataBunch(train_dl, valid_dl)


In [78]:
learn = Learner(data, model, loss_func=seq2seq_loss, metrics=seq2seq_acc)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,seq2seq_acc,time
0,1.931677,2.346272,0.125000,00:00
1,2.186545,2.278042,0.100000,00:00
2,1.987160,2.246424,0.175000,00:00
3,1.944333,2.194860,0.225000,00:00
4,1.931290,2.205694,0.250000,00:00


In [79]:
learn.save('5')

In [80]:
learn = Learner(data, model, loss_func=seq2seq_loss, metrics=seq2seq_acc)
learn.fit_one_cycle(5, 3e-3)
learn.save('5')

epoch,train_loss,valid_loss,seq2seq_acc,time
0,1.938486,2.121135,0.175000,00:00
1,1.960178,2.054171,0.329167,00:00
2,1.865897,2.228474,0.225000,00:00
3,1.860582,2.186399,0.275000,00:00
4,1.839330,2.157602,0.300000,00:00


In [81]:
def preds_acts(learn, ds_type=DatasetType.Train):
    learn.model.eval()
    rxs, rys, rzs, xs, ys, zs = [], [], [], [], [], []
    with torch.no_grad():
        for xb, yb in progress_bar(learn.dl(ds_type)):
            out = learn.model(xb)
            for x, y, z in zip(xb, yb, out):
                rxs.append(x)
                rys.append(y)
                preds = z.argmax(1)
                rzs.append(preds)
                xs.append(x.tolist())
                ys.append(y.tolist())
                zs.append(preds.tolist())
    return rxs, rys, rzs, xs, ys, zs

In [82]:
rxs,rys,rzs,xs,ys,zs = preds_acts(learn)

In [83]:
print(len(rxs), len(rys), len(rzs), len(xs), len(ys), len(zs))

8 8 8 8 8 8


In [84]:
idx = 3  # atau angka < panjang minimum dari list di atas

In [85]:
rx, ry, rz = rxs[idx], rys[idx], rzs[idx]
x, y, z = xs[idx], ys[idx], zs[idx]
rx, ry, rz

(tensor([4, 3, 8, 3, 2]), tensor([9, 6, 2, 6, 2]), tensor([9, 6, 1]))

In [86]:
def select_topk(outp, k=5):
    probs = F.softmax(outp,dim=-1)
    vals,idxs = probs.topk(k, dim=-1)
    return idxs[torch.randint(k, (1,))]

In [87]:
from random import choice

def select_nucleus(outp, p=0.5):
    probs = F.softmax(outp,dim=-1)
    idxs = torch.argsort(probs, descending=True)
    res,cumsum = [],0.
    for idx in idxs:
        res.append(idx)
        cumsum += probs[idx]
        if cumsum>p: return idxs.new_tensor([choice(res)])

In [88]:
def decode(self, inp):
    inp = inp[None]
    bs, sl = inp.size()
    hid,enc_out = self.encoder(bs, inp)
    dec_inp = inp.new_zeros(bs).long() + self.bos_idx
    enc_att = self.enc_att(enc_out)

    res = []
    for i in range(self.out_sl):
        hid, outp = self.decoder(dec_inp, hid, enc_att, enc_out)
        dec_inp = select_nucleus(outp[0], p=0.3)
#         dec_inp = select_topk(outp[0], k=2)
        res.append(dec_inp)
        if (dec_inp==self.pad_idx).all(): break
    return torch.cat(res)

In [89]:
print(x)
print(y)

[4, 3, 8, 3, 2]
[9, 6, 2, 6, 2]


In [90]:
def predict_with_decode(learn, x, y):
    learn.model.eval()
    with torch.no_grad():
        out = decode(learn.model, x)
    return x, y, out.argmax(-1)

In [91]:
x = rxs[0]
y = rys[0]

In [92]:
rx,ry,rz = predict_with_decode(learn, x, y)
rz

tensor(0)